# Qwen3模型架构与训练流程全解

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/188.png)

## 1 Qwen3开源模型概况

### 1.1 Dense + MoE两类架构、表现惊人

Qwen3系列模型由阿里巴巴开发，涵盖了多种规模的密集（Dense）和混合专家（MoE）模型，全部采用Apache 2.0开源许可证。

🔹 密集模型（Dense）

| 模型名称      | 参数量 | 层数 | 注意力头数（Q / KV） | 上下文长度 |
|---------------|--------|------|----------------------|------------|
| Qwen3-0.6B    | 0.6B   | 28   | 16 / 8               | 32K        |
| Qwen3-1.7B    | 1.7B   | 28   | 16 / 8               | 32K        |
| Qwen3-4B      | 4B     | 36   | 32 / 8               | 32K        |
| Qwen3-8B      | 8B     | 36   | 32 / 8               | 128K       |
| Qwen3-14B     | 14B    | 40   | 40 / 8               | 128K       |
| Qwen3-32B     | 32B    | 64   | 64 / 8               | 128K       |

🔸 混合专家模型（MoE）

| 模型名称         | 总参数量 | 激活参数量 | 层数 | 注意力头数<br>（Q / KV） | 专家数<br>（总数 / 激活数） | 上下文长度 |
|------------------|----------|------------|------|----------------------|-------------------------|------------|
| Qwen3-30B-A3B    | 30.5B    | 3.3B       | 48   | 32 / 4               | 128 / 8                 | 128K       |
| Qwen3-235B-A22B  | 235B     | 22B        | 94   | 64 / 4               | 128 / 8                 | 128K       |

> *从QKV头数来看，不难看出，Qwen3系列模型使用的是repeat_kv机制（重复kv缓存机制）*<br><br>
> *从专家数来看，使用了deepseek一致的小专家 * 大数量的方式，应当是EP方式运行*<br><br>
> *上下文长度中规中矩、YARN支持上下文拓展*<br><br>
> *惊人的是，比起架构、更多工作在“训练和后训练”，在Dense模型上完成了推理能力大幅提升的壮举*<br>

**<center><font color="red">Dense-4B模型在榜单上综合性能与DeepSeekv3模型相当、非常强悍！**</font>

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/189.png)

**<center><font color="red">MoE-30B模型在榜单上综合性能全面超越GPT4o！**</font>

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/190.png)

### 1.2 推理硬件与框架支持

- **推理硬件（官方推荐 + 测试结果）**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/187.png)

- **框架支持**

| 框架名称           | 是否支持 | 主要用途 |
|--------------------|----------|-----------|
| **Transformers**   | ✅ 支持  | 【通用】进行模型加载、推理和微调等通用操作 |
| **LoRA / PEFT**    | ✅ 支持  | 【微调】高效参数微调（如低秩适配） |
| **Llama Factory**  | ✅ 支持  | 【微调】多模型微调平台，支持指令微调、多任务训练等 |
| **DeepSpeed**      | ✅ 支持  | 【训练】大规模训练加速与优化，适用于分布式训练 |
| **vLLM**           | ✅ 支持  | 【推理】推理加速框架，提升解码速度和吞吐量 |
| **Llama.cpp**      | ✅ 支持  | 【推理】CPU 推理优化，支持在本地设备部署 |
| **LM Studio**      | ✅ 支持  | 【部署】在本地桌面端运行和测试模型 |
| **Ollama**         | ✅ 支持  | 【部署】本地命令行方式运行模型，简化部署流程 |
| **SGLang**           | ✅ 支持  | 【部署】实时大语言服务部署系统，高效处理批量请求 |
| **ModelScope (魔搭)** | ✅ 支持 | 【部署】阿里云官方模型开放平台，用于模型下载、部署、推理等 |

> *很显然，暂时不不支持多机多卡模型并行框架，如Megatron-LM*

### 1.3 全新概念：混合推理模型

Qwen3 引入了创新性的“混合推理模型”架构，**<font color="red">首次在一个统一的模型中融合了“快思考”（快速响应）与“慢思考”（深度推理）两种能力</font>**，旨在根据任务复杂度灵活调整推理策略，从而在性能和效率之间取得最佳平衡。

混合推理模型的核心理念是模拟人类在处理问题时的思维方式：

- **快思考模式（非思考模式）**：适用于简单任务，如日常对话、常识问答等，模型能够迅速给出答案，响应速度提升50%以上，显著降低算力消耗。

- **慢思考模式（思考模式）**：面对复杂问题，如数学推理、代码生成等，模型会进行逐步推理，生成更准确的答案。 

这种设计允许用户根据任务需求，在推理深度和响应速度之间进行权衡，实现性能与效率的完美平衡。

- **如何实现双模式切换？**

> 1. **显式控制**：用户可以通过指令（如 `/think` 或 `/no_think`）手动切换模式。<br><br>
> ![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/191.png)<br><br>
> ![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/192.png)

> 2. **API 参数设置**：在调用模型的 API 时，可以设置参数（如 `enable_thinking=True/False`）来控制默认模式。
> ![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/193.png)


> 3. **动态调整思考预算**：用户可以设置思考预算（如 token 数量）来限制模型的推理深度，从而控制算力消耗。

### 1.4 重训练、轻架构、工程仿照DeepSeek

- **预训练数据集暴增、训练流程复杂度大幅上升**

Qwen3 的数据集相比 Qwen2.5 得到了显著扩展。Qwen2.5 基于 18 万亿个 token 进行预训练，而 Qwen3 使用的数据量接近其两倍，约为 36 万亿个 token ，涵盖 119 种语言和方言 。为了构建这个大规模数据集，阿里不仅从网络上收集数据，还从类似 PDF 的文档中提取内容。Qwen3 使用 Qwen2.5-VL 从这些文档中提取文本，并利用 Qwen2.5 来提升所提取内容的质量。为了增加数学和代码相关的数据量，研发团队还使用 Qwen2.5-Math 和 Qwen2.5-Coder 生成了大量合成数据，包括教科书、问答对和代码片段。

>  1. **阶段一（S1）：基础语言能力构建**：在超过 30 万亿（30T）个 token 上进行预训练，Base上下文长度为 4K token，为模型提供基本的语言技能和通用知识。

>  2. **阶段二（S2，继续预训练）：知识密集型优化**：通过增加知识密集型数据（如 STEM、编程和推理任务）的比例，进一步训练模型。在额外的5万亿(5TB)个 token 上进行预训练，提升模型在专业领域的表现。

>  3. **阶段三（S3，继续预训练）：上下文能力扩展、长文本适应性训练**：使用高质量的长上下文数据，将上下文长度扩展到 32K token。确保模型能够有效地处理更长的输入，可能采用滑窗注意力、YARN拓展位置编码等操作。

- **后训练流程复杂度也大幅上升**

![](https://qianwen-res.oss-accelerate.aliyuncs.com/assets/blog/qwen3/post-training.png)

**Frontier Models（前沿模型、或者旗舰模型）**：这些是高性能、功能强大的模型，通常具有较大的参数量和更强的推理能力。它们的目标是实现最先进的性能，尤其是在复杂任务（如多步推理、长链思维等）上。这些模型参数量较大，计算资源需求高、专注于逐步推理（step-by-step reasoning）和复杂的任务处理、经过多阶段的精细训练，包括长链式思维冷启动、强化学习、思维模式融合等。

**Lightweight Models（轻量级模型）**：这些是小型化、计算效率高的模型，通常用于资源受限的场景或需要快速响应的应用中。它们的目标是在保证一定性能的同时，降低计算成本和延迟。这些模型参数量较小，计算资源需求低。专注于快速响应（rapid responses），适合部署在移动端或边缘设备上。通过蒸馏技术（distillation）从前沿模型中提取知识，从而在较小的模型规模下保持一定的性能。

下面是旗舰模型的训练流程 ↓

>  1. **长思维链冷启动（Long Chain-of-Thought Cold Start）**：使用多样的长思维链数据对模型进行微调，涵盖数学、代码、逻辑推理和 STEM 问题等多种任务和领域。

>  2. **长思维链强化学习（Long Chain-of-Thought Reinforcement Learning）**：利用基于规则的奖励机制（大概率GRPO或改进），增强模型的探索和钻研能力。

> 3. **思维模式融合（Thinking Mode Fusion，SFT）**：在包括长思维链数据和常用的指令微调数据的组合数据上对模型进行微调，将非思考模式整合到思考模型中，确保推理和快速响应能力的无缝结合。

> 4. **通用强化学习（General Reinforcement Learning）**：在包括指令遵循、格式遵循和 Agent 能力等在内的 20 多个通用领域的任务上应用强化学习，进一步增强模型的通用能力并纠正不良行为。

- **成本骤降、可能暗示有工程化优化细节**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/187.png)

> *更小的模型、更好的效果*<br><br>
> *DeepSeek617B需要3倍以上的显存才能进行推理（16*A800）*<br><br>
> *与DeepSeek671B模型对显卡的占用效率相当，很可能借鉴了DeepSeek模型的工程化技巧*

### 1.5 开源清单

以下是Qwen3模型开源内容的详细列表：

| 项目                   | 开源情况 | 说明 |
|------------------------|----------|------|
| 模型权重（BF16）       | ✅        | 全部模型提供BF16格式权重，适用于高性能训练和推理。 |
| 模型权重（FP8）        | ✅        | 全部模型提供FP8格式权重，显存需求缩小一半。<br>其中，8bitQwen3-30B-A3B支持ktransformer的调用<br>仅需Xeon4 + 1张4090即可调用<br>推理速度达到313tokens/s |
| 模型权重（4bit）        | 🔶 部分开源        | 可以通过ktrasnformer进行调用<br>包括Qwen3-30B-A3B模型和Qwen3-235B-A22B模型 <br>仅需Xeon4 + 1张4090即可调用<br>30B模型推理速度达到347.7tokens/s<br>235B模型推理速度达到115tokens/s|
| 基座模型（Base Model） | ✅   | Qwen3-30B-A3B-Base已开源，供研究和微调使用。 |
| 技术报告               | 🔶仅有概述 | 官方博客提供了极简版概述技术报告，但是细节严重缺失。 |
| 模型架构源码           | ❌        | 完全未公开源码、可找到CI代码 |
| 架构config配置   | ✅        | 全部模型提供config配置，可自行在huggingface查看 |
| 工程优化细节           | ❌        | 完全未公开、经推测应该大幅参考deepseek |
| 分词器（Tokenizer）    | ✅        | 全部模型的分词器已开源，支持多语言处理。 |
| 训练流程               | 🔶仅有概述 | 概述集训练流程的概述 | 

> 认识CI代码

**CI** 是 Continuous Integration（持续集成）的缩写，是现代软件工程中一套自动化构建和测试的流程工具链。在像 Hugging Face 这样的开源框架中，为了方便接入各类模型，他们通常会要求模型作者提交一个“**模块描述文件**”而不是手动写所有细节代码。例如：

- 模型的模块名（如 Qwen3Moe）
- 结构定义（如继承哪个类，用什么组件）
- 配置类名、tokenizer 名称
- 模型权重 URL、AutoClass 注册路径等

然后 Hugging Face 自己的 CI 系统会根据这些配置，通过预设的模板脚本自动生成：
- `modeling_qwen3_moe.py`（模型实现）
- `configuration_qwen3_moe.py`（模型配置）
- `tokenization_qwen3.py`（可选）

这个生成过程就是由 CI 自动执行的，不是人工一行一行敲的。这种做法在 Hugging Face、DeepSpeed、NVIDIA 开源仓库中都很常见。

**这是为了统一标准 + 降低出错概率**：

| 传统方式（手写）       | CI 自动化方式                    |
|----------------------|-------------------------------|
| 开发者手写所有模型代码 | 开发者只定义模块和结构             |
| 代码风格可能不统一     | 使用统一的模板格式输出             |
| 易出错、难测试         | 可以自动运行测试 +自动生成文档说明 |

在本次Qwen3开源过程中、我们能够拿到的是`modeling_qwen3_moe.py`脚本，这是 CI 系统根据 `modular_qwen3_moe.py` 这个“源模块文件”生成的，但是**开源仓库中并未公开 modular_qwen3_moe.py**，因为那是 Hugging Face 内部使用的描述式源码（metafile），所以我们现在只能看到“编译后的产物”，以次进一步了解模型架构，但是看不到“模板源文件”。

要注意，`modular_qwen3_moe.py`文件，并不是简单的“配置文件”，而是一个结构清晰的、带有完整 PyTorch 模块实现的“模块化源码”文件，它既能表达结构，也能还原参数和权重加载逻辑。它是这是 Hugging Face 推出的 “模块拼装式建模范式”。

在使用类似`modular_qwen3_moe.py`这样的文件、并利用CI 生成 `modeling_qwen3_moe.py`时，**<font color="red">必须保证生成后的代码加载后模型与训练时的模型结构完全一致**</font>，特别是：
- 层数、维度（hidden size、head 数等）
- attention、MLP、MoE 的配置参数
- forward 方法的输入输出方式
- 任何残差结构、Norm 位置

不过，这些都可以通过模块化源码定义精确表达，并非简单的 JSON 配置。

同时，**<font color="red">权重名字必须严格匹配**</font>，Hugging Face 的 `from_pretrained()` 方法依赖 **模块名 + 参数名精确匹配**（即 PyTorch 的 `state_dict` 结构）来加载。生成的模型类如果命名规范，参数注册正确，就能加载原始 `.bin` 或 `.safetensors` 权重。

除此之外、还要保证该模型能够 **<font color="red">兼容 tokenizer / config** </font> 。除了模型本体，还需要正确的 `Qwen3MoeConfig`，包括 vocab size、attention params 等、对应的 tokenizer（词表必须一致）、某些模型还需要自定义位置编码函数或 cache 支持模块。

## 2 Qwen3系列模型架构

### 2.1 基础架构

**Qwen3** 的模型架构大部分比较“经典”。整体设计融合了 **LLaMA-like** 和 **DeepSeek-like** 的特点，该架构从 **输入嵌入层（Embeddings）** 开始，通过多层堆叠的 Transformer 模块完成特征建模，最终通过 Softmax 输出预测结果。其中、每个 Transformer 模块由 **多查询注意力（Grouped Multi-Query Attention with KV Cache）**、**Feed Forward 模块（或 MoE）** 以及归一化和残差连接组成。

- **1. Input & Embeddings**：将离散的输入序列（如单词、字符或 Token）转换为连续向量表示，提供初始语义信息。作为整个模型的输入，提供每个 Token 的嵌入表示，用于后续的注意力和前馈处理。

- **2. RMS Norm（RMS归一化）**：对输入数据进行均方根归一化，稳定模型的训练，并加速收敛。RMS Norm 相比 Layer Norm 更高效，因为它只需计算每个样本的均方根值，而无需计算均值。

- **3. Multi Attention with repeated KV Cache**：实现分组自注意力机制，通过计算 Query、Key 和 Value 的相关性来捕获序列的上下文依赖，其中kv

- **4. Feed Forward（SwiGLU）与 MoE（Experts）**：对每个 Token 的表示单独进行非线性变换，增强模型的特征提取能力。其中FFN层使用门控激活单元，通过一组门控机制控制信息流动，能够捕获更复杂的特征，而MOE层则在多个专家模块中选择性激活一个或多个（通常互斥，即“Mutually Exclusive”），使模型在不同任务上具有更高的灵活性。

- **5. Residual Connection（残差连接）**：将输入直接添加到子模块的输出上，缓解梯度消失问题、使模型能够在学习额外特征时保留输入信息，从而更有效地学习深层次的表示。

- **6. Linear & Softmax**：线性层将 Transformer 模块的输出映射到词汇表大小，用于生成词分布。Softmax 将词分布转换为概率分布，完成预测。

在Qwen3开源的CI代码中，我们主要实现了一下的模块：

| 模块 | 功能 |
|------|------|
| **稀疏 MoE (Mixture-of-Experts)** | 支持 128 个专家，每个 token 最多使用 8 个，显著增强模型容量而不牺牲推理速度 |
| **GQA / MQA 注意力机制** | Key/Value Head 数量远小于 Query Head 数量，节省内存和计算资源 |
| **SwiGLU 激活函数** | 替代传统 ReLU，提升 FFN 表达能力 |
| **RoPE 位置编码** | 支持 32K 上下文长度，具有良好的外推性 |
| **RMSNorm 归一化** | 更轻量化的归一化方式，比 LayerNorm 更高效 |
| **KV Cache 管理机制** | 支持多种缓存策略（DynamicCache、SlidingWindowCache、StaticCache） |
| **支持多种下游任务头部** | 包括 Causal LM、Sequence Classification、Token Classification、Question Answering |

我们可以看到如下的类和函数——

1. **重要的类**

| 类名 | 功能 |
|------|------|
| **`Qwen3MoeAttention`** | 实现多头注意力机制（Multi-head Attention），支持 GQA/MQA（Grouped / Multi-Query Attention） |
| **`Qwen3MoeMLP`** | 基本的前馈网络（Feedforward Network），使用 SwiGLU 激活函数 |
| **`Qwen3MoeSparseMoeBlock`** | 实现稀疏 MoE（Mixture of Experts）模块，每个 token 只激活部分专家网络 |
| **`Qwen3MoeRMSNorm`** | RMS 归一化层 |
| **`Qwen3MoeDecoderLayer`** | Transformer 解码层，包含自注意力 + MLP/MoE 模块 |
| **`Qwen3MoeRotaryEmbedding`** | 实现 RoPE（旋转位置编码），支持超长上下文建模 |
| **`Qwen3MoeModel`** | 主干模型，由多个 DecoderLayer 组成，输出隐藏状态 |
| **`Qwen3MoeForCausalLM`** | 因果语言模型头部，用于文本生成任务 |
| **`Qwen3MoeForSequenceClassification`** | 序列分类任务头部，如情感分析、意图识别等 |
| **`Qwen3MoeForQuestionAnswering`** | 问答任务头部，用于抽取式问答（start/end logits） |
| **`Qwen3MoeForTokenClassification`** | Token 级别分类任务头部，如命名实体识别（NER） |
| **`Qwen3MoePreTrainedModel`** | 预训练模型基类，提供权重初始化、加载保存等通用功能 |

---

2. **重要函数**

> - **RoPE 相关函数**

| 函数名 | 功能 |
|--------|------|
| **`rotate_half(x)`** | 将输入张量的后半部分取反并与前半部分拼接，用于实现 RoPE 中的旋转操作 |
| **`apply_rotary_pos_emb(q, k, cos, sin)`** | 将 RoPE 编码应用到 Query 和 Key 向量上 |
| **`Qwen3MoeRotaryEmbedding.forward()`** | 生成并返回当前 batch 的 cos/sin 位置编码 |

> - **Attention 相关函数**

| 函数名 | 功能 |
|--------|------|
| **`repeat_kv(hidden_states, n_rep)`** | 对 Key/Value 进行重复以适应多头注意力中 GQA 的结构 |
| **`eager_attention_forward()`** | 实现标准的注意力计算（softmax(QK^T)V），不使用 flash attention |
| **`Qwen3MoeAttention.forward()`** | 完整的注意力模块前向传播，包括 Q/K/V 投影、RoPE 编码、KV 缓存等 |

> - **MoE 相关函数**

| 函数名 | 功能 |
|--------|------|
| **`Qwen3MoeSparseMoeBlock.forward()`** | 实现稀疏 MoE 模块的前向传播：路由选择 + 权重加权 + 多专家融合 |
| **`load_balancing_loss_func()`** | 计算 MoE 的负载均衡损失（Load Balancing Loss），防止某些专家被过度使用 |

> - **其他辅助函数**

| 函数名 | 功能 |
|--------|------|
| **`_compute_default_rope_parameters()`** | 根据配置参数计算 RoPE 所需的频率参数 |
| **`_update_causal_mask()`** | 更新因果掩码，防止未来信息“泄露”，支持滑动窗口等特性 |
| **`_prepare_4d_causal_attention_mask_with_cache_position()`** | 为 Attention 构造 4D 的因果掩码矩阵 |

### 2.2 Qwen3 MoE的分层稀疏调度机制

与传统MoE模型不同，Qwen3设置了`mlp_only_layers`参数允许使用者指定仅使用传统MLP的层序号，而`decoder_sparse_step`参数控制MoE层的插入间隔。例如——
> 设置 mlp_only_layers = [0,6] 时，模型将在第0、6层**启用MLP**，其余层保持MoE计算。<br><br>
> 设置 decoder_sparse_step = 3 时，则模型将在(idx + 1)能除尽3的层**启用MoE**，其余层保持MLP计算，例如，一共有12层，则在第2、5、8、11等层启用MoE计算，其余层保持MLP计算。<br><br>
> 设置 mlp_only_layers = [0,6]、且 decoder_sparse_step = 3时，两种机制混合但mlp_only_layers的优先级更高。

这段逻辑主要在`Qwen3MoeDecoderLayer.__init__() `中体现 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/195.png)

### 2.3 注意力机制引入QK标准化、取缔传统bias设计

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/197.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/Qwen3/196.png)

### 2.4 动态RoPE扩展与动态滑窗

旋转位置编码(RoPE)支持dynamic、yarn、llama3三种扩展模式！

在大语言模型（LLM）中，Rotary Position Embedding（RoPE）是一种常用的相对位置编码方法，用于增强模型对长序列的理解能力。然而，RoPE在处理超过预训练上下文长度的序列时，可能会出现性能下降的问题。为了解决这一挑战，研究人员提出了多种RoPE扩展策略，包括“dynamic”、“yarn”和“llama3”模式。

- Dynamic 模式（动态RoPE缩放）

**核心思想**：在推理阶段动态调整RoPE的缩放因子，以适应不同长度的输入序列

**特点**：

- 无需重新训练模型即可处理比预训练时更长的序列。
- 通过调整RoPE的频率参数，使得模型在长序列上的表现更加稳定。
- 适用于需要灵活处理不同长度输入的应用场景。

---

- YaRN 模式（Yet another RoPE extensioN）

**核心思想**：结合“NTK-aware”插值和注意力缩放技术，通过少量微调或仅在推理阶段调整，实现上下文窗口的高效扩展。

**特点**：

- 在不进行大规模重新训练的情况下，将上下文长度扩展到128k以上。
- 通过调整RoPE的插值策略，保留高频位置信息，提升模型在长序列上的表现。
- 适用于需要处理超长文本的任务，如长文档摘要和代码生成。

---

- Llama3 模式

**核心思想**：在Llama3模型中，对RoPE进行频率分段缩放，分别调整低频和高频分量，以优化模型在长序列上的性能。

**特点**：

- 引入`low_freq_factor`和`high_freq_factor`参数，分别控制低频和高频分量的缩放比例。
- 通过频率分段处理，减缓RoPE在长序列上的性能退化问题。
- 适用于需要在长序列上保持高性能的应用，如复杂对话系统和长文本生成。 
---


| 模式       | 是否需要微调 | 支持的上下文长度 | 适用场景                   |
|------------|--------------|------------------|----------------------------|
| Dynamic    | 否           | 2x 原始长度以上  | 灵活处理不同长度输入       |
| YaRN       | 可选         | 最长可达128k     | 超长文本处理，资源受限场景 |
| Llama3     | 否           | 依赖频率因子设置 | 长序列对话和文本生成       |